In [2]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import numpy as np

In [3]:
order_df=pd.read_csv('Data/orders.csv')
products_df=pd.read_csv('Data/products.csv')
order_product_prior_df=pd.read_csv('Data/order_products__prior.csv')
order_product_train_df=pd.read_csv('Data/order_products__train.csv')

In [4]:
train_users = order_df[(order_df.eval_set=="train")].user_id
test_users = order_df[(order_df.eval_set=="test")].user_id

In [5]:
train_set = order_df[order_df.user_id.isin(train_users)]
test_set = order_df[order_df.user_id.isin(test_users)]

In [6]:
test_users

38              3
44              4
53              6
96             11
102            12
            ...  
3420918    206202
3420929    206204
3421001    206206
3421018    206207
3421068    206208
Name: user_id, Length: 75000, dtype: int64

# TrainSet

In [7]:
# Merging order_df and order_products_prior_df
df_prior = pd.merge(order_df,order_product_prior_df,on='order_id')
df_train = pd.merge(order_df,order_product_train_df,on='order_id')

In [8]:
df = pd.concat([df_prior,df_train])
df.sort_values(by=['user_id','order_number'],inplace=True)

In [9]:
df

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
1,2539329,1,prior,1,2,8,NaN,14084,2,0
2,2539329,1,prior,1,2,8,NaN,12427,3,0
3,2539329,1,prior,1,2,8,NaN,26088,4,0
4,2539329,1,prior,1,2,8,NaN,26405,5,0
...,...,...,...,...,...,...,...,...,...,...
1384612,272231,206209,train,14,6,14,30.0,40603,4,0
1384613,272231,206209,train,14,6,14,30.0,15655,5,0
1384614,272231,206209,train,14,6,14,30.0,42606,6,0
1384615,272231,206209,train,14,6,14,30.0,37966,7,0


In [10]:
df_train = df[df["user_id"].isin(train_users)]
df_test = df[df["user_id"].isin(test_users)]

In [11]:
df_train

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
1,2539329,1,prior,1,2,8,NaN,14084,2,0
2,2539329,1,prior,1,2,8,NaN,12427,3,0
3,2539329,1,prior,1,2,8,NaN,26088,4,0
4,2539329,1,prior,1,2,8,NaN,26405,5,0
...,...,...,...,...,...,...,...,...,...,...
1384612,272231,206209,train,14,6,14,30.0,40603,4,0
1384613,272231,206209,train,14,6,14,30.0,15655,5,0
1384614,272231,206209,train,14,6,14,30.0,42606,6,0
1384615,272231,206209,train,14,6,14,30.0,37966,7,0


In [12]:
df_test

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
254,1374495,3,prior,1,1,14,NaN,9387,1,0
255,1374495,3,prior,1,1,14,NaN,17668,2,0
256,1374495,3,prior,1,1,14,NaN,15143,3,0
257,1374495,3,prior,1,1,14,NaN,16797,4,0
258,1374495,3,prior,1,1,14,NaN,39190,5,0
...,...,...,...,...,...,...,...,...,...,...
32434355,1882108,206208,prior,49,1,22,7.0,2643,13,1
32434356,1882108,206208,prior,49,1,22,7.0,21405,14,0
32434357,1882108,206208,prior,49,1,22,7.0,7632,15,0
32434358,1882108,206208,prior,49,1,22,7.0,1942,16,0


## Selecting only top

In [13]:
product_id = order_product_prior_df["product_id"].value_counts()[:500].index.tolist()

In [14]:
df_train = df_train[df_train["product_id"].isin(product_id)]

In [15]:
df_train["user_id"].value_counts()

127158    1414
160106    1386
182401    1383
100787    1355
166786    1342
          ... 
174359       1
48783        1
100343       1
149376       1
178749       1
Name: user_id, Length: 129645, dtype: int64

In [16]:
user_id = df_train["user_id"].value_counts()[:3000].index.tolist()

In [17]:
df_train = df_train[df_train["user_id"].isin(user_id)]

In [18]:
df_train

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
16837,1207960,140,prior,1,3,17,NaN,18370,4,0
16838,2093304,140,prior,2,6,10,3.0,19057,1,0
16840,2093304,140,prior,2,6,10,3.0,11520,3,0
16845,2093304,140,prior,2,6,10,3.0,21616,8,0
16849,2254976,140,prior,3,2,11,3.0,11520,3,1
...,...,...,...,...,...,...,...,...,...,...
32411637,2677849,206053,prior,60,1,13,19.0,5876,30,1
1383435,395225,206053,train,61,4,17,3.0,19678,1,1
1383436,395225,206053,train,61,4,17,3.0,26209,2,1
1383438,395225,206053,train,61,4,17,3.0,47672,4,1


In [19]:
df_train.nunique()

order_id                  192751
user_id                     3000
eval_set                       2
order_number                 100
order_dow                      7
order_hour_of_day             24
days_since_prior_order        31
product_id                   500
add_to_cart_order            113
reordered                      2
dtype: int64

In [20]:
len(df_train[df_train.eval_set=='train'].user_id.unique().tolist())

2956

In [21]:
train_user = df_train[df_train.eval_set=='train'].user_id.unique().tolist()

In [22]:
df_train = df_train[df_train["user_id"].isin(train_user)]

In [23]:
df_train.nunique()

order_id                  189395
user_id                     2956
eval_set                       2
order_number                 100
order_dow                      7
order_hour_of_day             24
days_since_prior_order        31
product_id                   500
add_to_cart_order            113
reordered                      2
dtype: int64

In [24]:
len(df_train[df_train.eval_set=='train'].user_id.unique().tolist())

2956

## Train

In [25]:
df_train

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
16837,1207960,140,prior,1,3,17,NaN,18370,4,0
16838,2093304,140,prior,2,6,10,3.0,19057,1,0
16840,2093304,140,prior,2,6,10,3.0,11520,3,0
16845,2093304,140,prior,2,6,10,3.0,21616,8,0
16849,2254976,140,prior,3,2,11,3.0,11520,3,1
...,...,...,...,...,...,...,...,...,...,...
32411637,2677849,206053,prior,60,1,13,19.0,5876,30,1
1383435,395225,206053,train,61,4,17,3.0,19678,1,1
1383436,395225,206053,train,61,4,17,3.0,26209,2,1
1383438,395225,206053,train,61,4,17,3.0,47672,4,1


In [26]:
df_train.drop(columns=['reordered','add_to_cart_order'],inplace=True)

C:\Users\muhammad.sabih\Anaconda3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [27]:
df_train

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id
16837,1207960,140,prior,1,3,17,NaN,18370
16838,2093304,140,prior,2,6,10,3.0,19057
16840,2093304,140,prior,2,6,10,3.0,11520
16845,2093304,140,prior,2,6,10,3.0,21616
16849,2254976,140,prior,3,2,11,3.0,11520
...,...,...,...,...,...,...,...,...
32411637,2677849,206053,prior,60,1,13,19.0,5876
1383435,395225,206053,train,61,4,17,3.0,19678
1383436,395225,206053,train,61,4,17,3.0,26209
1383438,395225,206053,train,61,4,17,3.0,47672


In [28]:
#temp_table1 = df_train.groupby(['user_id','order_number','order_id'])['product_id'].apply(list)

In [29]:
df_train.nunique()

order_id                  189395
user_id                     2956
eval_set                       2
order_number                 100
order_dow                      7
order_hour_of_day             24
days_since_prior_order        31
product_id                   500
dtype: int64

In [30]:
df_train = pd.get_dummies(df_train, columns=['product_id'],prefix='', prefix_sep='')

In [31]:
df_cols = df_train[['user_id','order_number','order_id','order_dow','order_hour_of_day','days_since_prior_order']]

In [32]:
df_cols.drop_duplicates()

,user_id,order_number,order_id,order_dow,order_hour_of_day,days_since_prior_order
16837,140,1,1207960,3,17,NaN
16838,140,2,2093304,6,10,3.0
16849,140,3,2254976,2,11,3.0
16869,140,4,3369359,0,12,5.0
16883,140,6,726285,1,17,5.0
...,...,...,...,...,...,...
32411566,206053,57,3176305,4,13,5.0
32411580,206053,58,2759093,0,10,3.0
32411593,206053,59,1296202,3,16,3.0
32411608,206053,60,2677849,1,13,19.0


In [33]:
df_train.drop(columns=['order_dow','order_hour_of_day','days_since_prior_order'],inplace=True)

In [34]:
df_train

,order_id,user_id,eval_set,order_number,45,196,260,329,432,651,...,48679,48745,48775,49075,49175,49191,49235,49383,49520,49683
16837,1207960,140,prior,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16838,2093304,140,prior,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16840,2093304,140,prior,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16845,2093304,140,prior,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16849,2254976,140,prior,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32411637,2677849,206053,prior,60,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1383435,395225,206053,train,61,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1383436,395225,206053,train,61,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1383438,395225,206053,train,61,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
df_train = df_train.groupby(['user_id','order_number','order_id']).sum()

In [36]:
df_train

45  196  260  329  432  651  890  1025  1158  \
user_id order_number order_id                                                 
140     1            1207960    0    0    0    0    0    0    0     0     0   
        2            2093304    0    0    0    0    0    0    0     0     0   
        3            2254976    0    0    0    0    0    0    0     0     0   
        4            3369359    0    0    0    0    0    0    0     0     0   
        6            726285     0    0    0    0    0    0    0     0     0   
...                            ..  ...  ...  ...  ...  ...  ...   ...   ...   
206053  57           3176305    0    0    0    0    0    0    0     0     0   
        58           2759093    0    0    0    0    0    0    0     0     0   
        59           1296202    0    0    0    0    0    0    0     0     0   
        60           2677849    0    0    0    0    0    0    0     0     0   
        61           395225     0    0    0    0    0    0    0     0     0   

                               1194  ...  48679  48745  48775  49075  49175  \
user_id order_number order_id        ...                                      
140     1            1207960      0  ...      0      0      0      0      0   
        2            2093304      0  ...      0      0      0      0      0   
        3            2254976      0  ...      0      0      0      0      0   
        4            3369359      0  ...      0      0      0      0      0   
        6            726285       0  ...      0      0      0      0      0   
...                             ...  ...    ...    ...    ...    ...    ...   
206053  57           3176305      0  ...      0      0      0      0      0   
        58           2759093      0  ...      1      0      0      0      0   
        59           1296202      0  ...      0      0      0      0      0   
        60           2677849      0  ...      0      0      0      0      0   
        61           395225       0  ...      0      0      0      0      0   

                               49191  49235  49383  49520  49683  
user_id order_number order_id                                     
140     1            1207960       0      0      0      0      0  
        2            2093304       0      0      0      0      0  
        3            2254976       0      0      0      0      0  
        4            3369359       0      0      0      0      0  
        6            726285        0      0      0      0      0  
...                              ...    ...    ...    ...    ...  
206053  57           3176305       0      0      0      0      0  
        58           2759093       0      0      0      0      0  
        59           1296202       0      0      0      0      0  
        60           2677849       0      0      0      0      0  
        61           395225        0      0      0      0      0  

[189395 rows x 500 columns]

In [37]:
df_train.reset_index(inplace=True)

In [38]:
df_train = pd.merge(df_train,df_cols,on=['user_id','order_id','order_number'])

In [39]:
df_train.drop_duplicates(inplace=True)

In [40]:
df_train

,user_id,order_number,order_id,45,196,260,329,432,651,890,...,49075,49175,49191,49235,49383,49520,49683,order_dow,order_hour_of_day,days_since_prior_order
0,140,1,1207960,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,17,NaN
1,140,2,2093304,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,6,10,3.0
4,140,3,2254976,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,11,3.0
13,140,4,3369359,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,12,5.0
21,140,6,726285,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,17,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1613001,206053,57,3176305,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,4,13,5.0
1613011,206053,58,2759093,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,10,3.0
1613021,206053,59,1296202,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,16,3.0
1613032,206053,60,2677849,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,13,19.0


In [41]:
df_train.shape

(189395, 506)

In [42]:
#df_train.to_csv('DataWithoutPadding.csv')

## Padding

In [43]:
df_train.user_id.value_counts()

161043    100
203256    100
54259     100
16600     100
75664     100
         ... 
83513      23
132632     21
126266     20
140539     20
134391     19
Name: user_id, Length: 2956, dtype: int64

In [44]:
max_rows=df_train.user_id.value_counts().iloc[0]
max_rows

100

In [45]:
for user in user_id:
    length = len(df_train[df_train["user_id"] == user])    
    padding = max_rows - length
    padding_list = []
    [padding_list.append({'user_id':user,'order_number':0}) for i in range(0,padding)]        
    df_train = df_train.append(padding_list, ignore_index=True)

In [46]:
df_train

,user_id,order_number,order_id,45,196,260,329,432,651,890,...,49075,49175,49191,49235,49383,49520,49683,order_dow,order_hour_of_day,days_since_prior_order
0,140,1,1207960.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,17.0,NaN
1,140,2,2093304.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,10.0,3.0
2,140,3,2254976.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,11.0,3.0
3,140,4,3369359.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,5.0
4,140,6,726285.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,17.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,89212,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
299996,89212,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
299997,89212,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
299998,89212,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
len(df_train)

300000

In [48]:
df_train.sort_values(by=['user_id','order_number'],inplace=True)

In [49]:
df_train[df_train.user_id==313]

,user_id,order_number,order_id,45,196,260,329,432,651,890,...,49075,49175,49191,49235,49383,49520,49683,order_dow,order_hour_of_day,days_since_prior_order
214777,313,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
214778,313,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
214779,313,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
214780,313,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
214781,313,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,313,96,2375876.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,6.0,1.0
343,313,97,1860207.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,12.0,0.0
344,313,98,3341750.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,3.0,2.0
345,313,99,1380730.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,11.0,0.0


In [50]:
df_train.to_csv('Prepared_Data/DataWithPadding.csv')